In [1]:
import numpy as np
import pandas as pd 
import torch
from torch import nn
time = np.loadtxt("../Data/AAL/100408_timeseries.txt")

In [2]:
torch.__version__

'2.1.1+cu121'

In [3]:
time.shape

(1200, 116)

In [4]:
from Dataset import TimeSDataset
Dataset = TimeSDataset("../Data/AAL/","Gender",label_path="../Data/Behavioral-HCP.csv")

In [5]:
data , label = next(iter(Dataset))
data = torch.stack([data,data])
data.shape , label

(torch.Size([2, 1200, 116]), tensor(1))

In [6]:
label.shape

torch.Size([])

In [7]:
from Config import Config
from SplitWindows import SplitWindows
import torch
SplitedSeries = SplitWindows(data)

In [8]:
SplitedSeries.shape   # ( B , T , V , P )

torch.Size([2, 105, 116, 20])

In [9]:
Config.T

105

In [10]:
from AttentionArea import TemporalAttention , SpatialAttention
TAttention = TemporalAttention (8 ,Config.V * Config.K_E , 32 )

In [11]:
Temporal_output , weights = TAttention(SplitedSeries)
# (B , T , V , K_E)
Temporal_output.shape , weights.shape

(torch.Size([2, 105, 116, 40]), torch.Size([2, 105, 105]))

In [12]:
# Spatial_output = SAttention(Temporal_output)

In [13]:
Config.V * Config.K_E 


4640

In [14]:
from AttentionArea import DynamicMatrix
DM = DynamicMatrix()
weights = DM(Temporal_output)
weights.shape

torch.Size([2, 105, 116, 116])

In [15]:
from AttentionArea import SpatialAttention
ST = SpatialAttention()
SpatialOut = ST(Temporal_output)
SpatialOut.shape

torch.Size([2, 105, 116, 40])

In [16]:
from torch_geometric.nn import GCNConv 

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
weights[1][1].nonzero()

tensor([[  0,   1],
        [  0,   8],
        [  0,  12],
        ...,
        [115,  86],
        [115,  93],
        [115,  98]])

In [18]:
weights[1][1][0][16]

tensor(0., grad_fn=<SelectBackward0>)

In [19]:
import torch
from torch_geometric.nn import GCNConv

# Assuming x is your node features with shape (B, T, V, P)
# and adj is your weighted adjacency matrix with shape (B, T, V, V)
Batch = []

gcn_layer = GCNConv(in_channels=Config.K_E, out_channels=Config.K_F)

# Iterate over each graph in the batch
for i in range(Config.BATCH_SIZE):
    # Extract the adjacency matrix for the current graph
    adj_i = weights[i]  # Shape (T, V, V)
    out = []
    for t in range(Config.T):
        # Get the edge indices and weights for the current time step
        edge_index = adj_i[t].nonzero(as_tuple=False).T  # Shape (2, E)
        edge_weights = adj_i[t][edge_index[0], edge_index[1]]  # Shape (E,)
        # print(edge_index)
        # Select the corresponding node features for the current time step
        x_t = SpatialOut[i][t]
        # Apply the GCN layer
        x_t = gcn_layer(x_t, edge_index, edge_weight=edge_weights)
        # Store the output for the current time step
        out.append(x_t)
    Batch.append(torch.stack(out))

# Stack the output to get the final representation
out = torch.stack(Batch)  # Shape will depend on how you structure the output

In [20]:
out.shape

torch.Size([2, 105, 116, 40])